# Regla de Bayes i aplicació a la classificació de text

En aquest pràctica introduïrem dos conceptes que no vem arribar a veure a les classes de teoria:

* la regla de Bayes [Casella & Berger 1.3.5],
* i la independència condicional

i els aplicarem per desenvolupar un classificador de text binari (dues classes).

La pràctica comença amb una mica de teoria, conté alguns exercicis de paper i llapis i finalment alguns exercicis computacionals. El codi el podeu desenvolupar en R o en Python, com us vingui millor. 

Els resultats dels exercicis els haureu d'entregar pel **dia 5/20/2020 abans de les 9:00**. Durant la classe d'aquest dia us en donaré la resolució.

Recordeu que heu d'entregar el resultat de la pràctica per parelles (amb excepció d'algun triplet en cas de ser senars). No oblideu incloure el vostre Nom i NIU a la tramesa. 

## Regla de Bayes

Donada una partició $A_1, A_2, \cdots, A_N$ d'$\Omega$, la regla de Bayes ens permet calcular la probabilitat d'un esdeveniment $A_i$ donat un altre esdeveniment $B$ com [Casella & Berger 1.3.5]:

$P(A_i|B) = \frac{P(B|A_i)P(A_i)}{\sum_{j=1}^NP(B|A_j)P(A_j)}$

Normalment anomenarem:

* $P(A_i|B)$ la probabilitat *posterior* d'$A_i$ havent observat $B$,
* $P(B|A_i)$ la *versemblança* de B condicionat a $A_i$
* $P(A_i)$ l'*a priori* d'$A_i$

La fórmula és fàcil de demostrar a partir de l'identitat $P(A|B) P(B) = P(A \cap B) = P(B|A) P(A)$.

Vegem amb un exercici d'exemple l'importància d'aquesta fórmula.

### Exercici 1

Considereu un test mèdic per una certa patologia. El test té dos resultats possibles: $\left\{+, -\right\}$. Per altra banda, una persona d'una població pot tenir o no tenir la patologia, que denotarem per $\left\{\mbox{Patologia}, \overline{\mbox{Patologia}}\right\}$.

Suposem que el test no és perfecte i per tant comet errors. Per exemple:

* $P(+ | \overline{\mbox{Patologia}}) = 0.1$ (fals positiu)
* $P(- | {\mbox{Patologia}}) = 0.05$ (fals negatiu)

Per altra banda, sabem que la prevalència de la patologia en la població és molt baixa, posem
$P({\mbox{Patologia}}) = 0.001$. 

Si ens fem el test i ens dona positiu, quina és la probabilitat de que realment sofrim la patologia?

## Classificació 

La regla de Bayes es pot fer servir per dissenyar classificadors. Per exemple, considereu els esdeveniments definits sobre una població de correus electrònics:

* $A_1 = \mbox{SPAM}$: El correu és spam (no desitjat)
* $A_2 = \overline{\mbox{SPAM}}$: El correu no és spam (també anomenat "ham")
* $B$: L'esdeveniment corresponent a la presència o absència de $K$ certes paraules en el text del correu. Per exemple $B=W_1 \cap W_2^c\cap W_3$ es correspondria amb un correu el text del qual conté les paraules $W_1$ i $W_3$ i no conté la paraula $W_2$.

Un classificador de correus electrònics és una funció que pren el text corresponent a $B$ i retorna $\left\{\mbox{SPAM}, \overline{\mbox{SPAM}}\right\}$. Considerem la següent funció:

$f(B) = \left\{\begin{array}{cc}\mbox{SPAM} & \mbox{ si } P(A_1|B) > P(A_2|B) \\\overline{\mbox{SPAM}} & \mbox{altrament} \end{array}\right.$


### Exercici 2

Perquè té sentit una tal proposta?

### Exercici 3

En realitat, gràcies a la regla de Bayes, podem definir el nostre classificador en funció només del text del correu. Per fer-ho aplicarem la regla de Bayes per obtenir:

$f(B) = \left\{\begin{array}{cc}\mbox{SPAM} & \mbox{ si } P(B|\mbox{SPAM})P(\mbox{SPAM}) > P(B|\overline{\mbox{SPAM}})P(\overline{\mbox{SPAM}}) \\\overline{\mbox{SPAM}} & \mbox{altrament} \end{array}\right.$

Anem a provar el nostre classificador. Per tal d'implementar-lo, hauriem de tenir coneixement de les quantitats $P(B|A_1), P(A_1), P(B|A_2), P(A_2)$ per qualsevol B, que no coneixem. Per tant, les haurem d'**estimar** a partir de **dades d'entrenament** (training data).

Per començar, ens imaginarem que volem classificar els correus només en funció de la presència o absència de dues paraules: "prince", "USD". L'esdeveniment $B$ per tant ens indica si alguna cap o totes aquestes paraules són presents en un determinat correu. Per exemple el correu amb text "I am a Nigerian prince - send me 1000USD" es correspon amb l'esdeveniment $B=\mbox{prince} \cap \mbox{USD}$.

Construïu un conjunt de dades d'entrenament "de joguina" amb 10 files. Us en dono dues d'exemple:

|  Etiqueta | Correu                             |  "prince" | "USD"  |
|-----------|------------------------------------|-----------|--------|
|  $\mbox{SPAM}$     | I am a Nigerian prince - send me 1000USD  |      1    |   1    |
|  $\overline{\mbox{SPAM}}$  | Hey Arnau - how have you been?     |      0    |   0    |

### Exercici 4

Amb el vostre conjunt de dades, estimeu les quantitats $P(B|A_1), P(A_1), P(B|A_2), P(A_2)$.
Pista: L'estimador més simple de $P(B = \mbox{prince}^c \cap \mbox{USD}^c|A_1)$ seria

$\hat{P}(B = \mbox{prince}^c \cap \mbox{USD}^c |A_1) = \frac{\mbox{Nombre de correus en que no apareixen "prince" ni "USD"}}{\mbox{Nombre de correus}}$

Fantàstic! Un cop ja teniu estimat $P(B|A_1), P(A_1), P(B|A_2), P(A_2)$ ja teniu "entrenat" el vostre classificador de Bayes, que podeu fer servir amb qualsevol correu nou, només mirant si conté aquestes dues paraules clau

## Classificació per Bayes naïf

Evidentment que el model que hem dissenyat fins ara era una mica limitat: tots ens podem imaginar varietats de correus de spam que no serien identificats pel nostre primer classificador. Això es deu en part al fet que fins ara només ens hem basat en dues paraules per prendre la nostra decisió.

Sorprenentment però, l'idea bàsica que hem fet servir dona lloc a un classificador anomenat de Bayes Naïf, que fins fa relativament poc (10-15 anys) era competitiu amb els millors classificadors d'aprentissatge automàtic per problemes d'aquest tipus.

Considereu doncs el cas on volem construir el nostre classificador fent servir un vocabulari més ampli. El problema de fer servir un vocabulari més ampli és que haurem d'estimar $P(B|A_1)$ i $P(B|A_2)$ per moltes possibles combinacions de $B$. 

### Exercici 5

Si abans, quan feiem servir 2 paraules, hem hagut d'estimar les 4 possibles combinacions de B, si en fem servir $N$, quantes combinacions hi haurà?

## La naiveté

L'idea fonamental d'aquest mètode és la següent aproximació:

$P\left(B = \cap_{i=1}^K W_i | A\right) \approx P(W_1|A)P(W_2|A)\cdots P(W_K|A)$

La igualtat seria certa només en el cas que els esdeveniments $W_i$ fóssin independents condicionals a $A$.

Per tant, enlloc d'haver d'estimar $P\left(B = \cap_{i=1}^K W_i | A\right)$, cosa complicada com hem vist en l'Exercici 5, només haurem d'estimar $P(W_1|A), P(W_2|A), \cdots P(W_K|A)$. Per posar-ho en termes de l'exemple de l'Exercici 4, amb aquesta aproximació calculariem $\hat{P}(B = \mbox{prince}^c \cap \mbox{USD}^c |A_1)$ com:

$\hat{P}(B = \mbox{prince}^c \cap \mbox{USD}^c |A_1) = \frac{\mbox{Nombre de correus en que no apareixen "prince" }}{\mbox{Nombre de correus}} \times \frac{\mbox{Nombre de correus en que no apareixen "USD" }}{\mbox{Nombre de correus}}$

Som-hi doncs. Primer ens descarregarem un conjunt de dades de veritat. Visiteu [aquesta pàgina i premeu "Donwload"](https://www.kaggle.com/venky73/spam-mails-dataset).

Carregarem les dades amb pandas (o en la vostra llibreria preferida d'R):

In [12]:
import pandas as pd

dataset = pd.read_csv('/Users/arnau.tibau/Downloads/spam_ham_dataset 2.csv')

dataset.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


### Exercici 6

1. Separeu el conjunt de dades en un conjunt d'entrenament amb les 15000 primeres files i un de test amb les files restants
2. Implementeu una funció `estimate_word_frequency(data, word)` que prengui com a input el conjunt de dades d'entrenament i una cadena de caràcters "word", i retorni la frequència de la paraula en cada classe (és a dir, un estimador de $P(\mbox{word}|\mbox{SPAM})$ i $P(\mbox{word}|\overline{\mbox{SPAM}})$ )
3. Apliqueu aquest a funció a les paraules ["Subject:", "company", "http", "information", "enron", "gas", "deal", "meter", "forwarded"] i emmagatzemeu els resultats en un diccionary `word_frequencies` tal que per cada paraula hi teniu la seva frequència en SPAM i HAM.
4. Implementeu la funció `estimate_spam_frequency(data)` que pren com a input el conjunt de dades i retorna una estimació de $P(\mbox{SPAM})$.
5. Implementeu una funció `estimate_likelihood(text, word_frequencies)` que pren com a input el text d'un correu (`text`), i l'estimació de freqüències de cada paraula (`word_frequencies`) i retorna una estimació de les versemblances $P(B|\overline{\mbox{SPAM}})$ i $P(B|\mbox{SPAM})$ fent servir l'aproximació de Bayes naïf.
6. Implementeu la funció `naive_bayes(text, word_frequencies, spam_frequency)` que pren com a input el text d'un correu (`text`), l'estimació de la freqüència d'spam (`spam_frequency`) i l'estimació de freqüències de cada paraula (`word_frequencies`) i calcula $P(B|\mbox{SPAM})P(\mbox{SPAM}) > P(B|\overline{\mbox{SPAM}})P(\overline{\mbox{SPAM}})$ fent servir l'aproximació de Bayes Naïf. Per fer-ho, aprofiteu la funció `estimate_likelihood(text, word_frequencies)` que acabeu d'implementar.
7. Proveu el vostre classificador amb alguns exemples del conjunt d'entrenament, i alguns del conjunt de test. Què tal funciona? Classifica algun exemple correctament? 
8. [OPCIONAL] Què podriem fer per millorar-lo?

In [13]:
# Algunes funcions auxiliars que us podrien ser útils

def word_count(text_list, min_char=3):
    """
    Retorna el nombre de vegades que apareix cada paraula d'al menys `min_char`
    caràcters a la llista de texts `text_list`
    """
    counts = {}
    for text in text_list:
        for word in text.split():
            if len(word) < min_char:
                continue
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
            
    return sorted(counts.items(), key=lambda x: -x[1])